<a href="https://colab.research.google.com/github/SuzuT1212/hokudai_sylbs/blob/main_suzu/%E6%88%90%E7%B8%BE%E5%88%86%E5%B8%83%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)
driver.get("http://educate.academic.hokudai.ac.jp/seiseki/GradeDistResult11.aspx")
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")

In [ ]:
from selenium.webdriver.support.ui import Select
# 年度の指定
year_element = driver.find_element_by_id('ddlTerm')
year_select_element = Select(year_element)
year_select_element.select_by_value('20201')

# 課程区分の指定
degree_element = driver.find_element_by_id('ddlDiv')
degree_select_element = Select(degree_element)
degree_select_element.select_by_value('02')

# 開講学部の指定
fac_element = driver.find_element_by_id('ddlFac')
fac_select_element = Select(fac_element)
fac_select_element.select_by_value('00')

# データ種別の指定
kinds_element = driver.find_element_by_id('ddlDataKind')
kinds_select_element = Select(kinds_element)
kinds_select_element.select_by_value('1')

# 検索をクリック
driver.find_element_by_xpath('//*[@id="btnSerch"]').click()

# 表示件数を全件にする
exp_element = driver.find_element_by_id('ddlLine_ddl')
exp_select_element = Select(exp_element)
exp_select_element.select_by_value('0')

In [ ]:
# htmlを取得しスクレイピングしていく
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
# テーブルを指定
table = soup.find('table', id="gvResult").tbody
rows = table.find_all('tr')
# print(rows[0])

In [ ]:
# dateframeを作るためにcolumn作成
columns = [v.text for v in rows[0].find_all('th')]

df = pd.DataFrame(columns=columns)

for i in range(len(rows)):
  # 全ての<td>タグ（セルデータ）を取得しtdsに格納、リスト化
  tds = rows[i].find_all('td')
  # tdsのデータ数がカラム数に一致しない場合（ブランク）などは排除し、
  if len(tds) == len(columns):
    # （ある行成分の）全セルデータをテキスト成分としてvaluesに格納、リスト化
    values = [ td.text.replace('\n', '').replace('\xa0', ' ') for td in tds ]
    # valuesをpd.seriesデータに変換、データフレームに結合
    df = df.append(pd.Series(values, index=columns), ignore_index= True)

,,授業科目名,講義題目名,クラス,担当教員名,履修者数,A+(%),A(%),A-(%),B+(%),B(%),B-(%),C+(%),C(%),D(%),D-(%),F(%),GPA
0,1,思索と言語,アイヌ語を通して文化を学ぶ(1),基礎1-53組,北原 次郎太,209,3.8,9.1,11.0,14.4,14.8,10.5,10.5,10.5,3.8,11.5,0.0,2.71
1,2,思索と言語,ヨーロッパの言語誌―英語とその仲間たち,基礎1-53組,清水 誠,134,3.0,10.4,9.0,50.0,0.0,13.4,3.0,2.2,0.0,0.0,9.0,3
2,3,思索と言語,中国哲学と論理学,基礎1-53組,近藤 浩之,48,18.8,60.4,6.3,0.0,4.2,0.0,0.0,2.1,0.0,0.0,8.3,3.62
3,4,思索と言語,九鬼周造を読む,"基礎1-15,53組",村松 正隆,15,13.3,6.7,26.7,13.3,13.3,6.7,0.0,0.0,0.0,0.0,20.0,2.85
4,5,思索と言語,社会哲学入門,基礎1-53組,見附 陽介,87,5.7,13.8,5.7,10.3,12.6,13.8,9.2,8.0,1.1,1.1,18.4,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,808,英語再履修,,基礎1-52組,天田 顕徳,19,0.0,15.8,21.1,15.8,15.8,5.3,0.0,0.0,0.0,0.0,26.3,2.55
808,809,英語再履修,,基礎1-52組,鈴木 志のぶ,12,0.0,0.0,8.3,16.7,0.0,16.7,8.3,16.7,8.3,8.3,16.7,1.98
809,810,英語再履修,,基礎1-52組,岡本 亮輔,27,3.7,11.1,0.0,37.0,11.1,18.5,3.7,0.0,0.0,0.0,14.8,2.74
810,811,英語再履修,合計,,(講義題目数7),172,1.2,7.6,14.0,15.1,9.9,9.9,3.5,11.0,2.9,6.4,18.6,2.31


Datebeseに変換

In [ ]:
import sqlite3
from sqlalchemy import create_engine

engine = create_engine('sqlite:///trg.db', echo=False,encoding="utf-8")
df.to_sql("df", engine)